In [22]:
from Juliet_DB_loader import TestCase, Folder, SubFolder, sessionmaker, engine
from sqlalchemy import func
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()


In [25]:
test_cases = session.query(TestCase).filter_by(id=4).first()
line_content = dict()
lines = test_cases.file_content.split('\n')
for line_num, line in enumerate(lines, start=1):
    line_content[line_num] = line

# Print the dictionary vertically
for line_num, line in line_content.items():
    print(f"{line_num}: {line}")

1: /* TEMPLATE GENERATED TESTCASE FILE
2: Filename: CWE114_Process_Control__w32_char_connect_socket_01.c
3: Label Definition File: CWE114_Process_Control__w32.label.xml
4: Template File: sources-sink-01.tmpl.c
5: */
6: /*
7:  * @description
8:  * CWE: 114 Process Control
9:  * BadSource: connect_socket Read data using a connect socket (client side)
10:  * GoodSource: Hard code the full pathname to the library
11:  * Sink:
12:  *    BadSink : Load a dynamic link library
13:  * Flow Variant: 01 Baseline
14:  *
15:  * */
16: 
17: #include "std_testcase.h"
18: 
19: #include <wchar.h>
20: 
21: #ifdef _WIN32
22: #include <winsock2.h>
23: #include <windows.h>
24: #include <direct.h>
25: #pragma comment(lib, "ws2_32") /* include ws2_32.lib when linking */
26: #define CLOSE_SOCKET closesocket
27: #else /* NOT _WIN32 */
28: #include <sys/types.h>
29: #include <sys/socket.h>
30: #include <netinet/in.h>
31: #include <arpa/inet.h>
32: #include <unistd.h>
33: #define INVALID_SOCKET -1
34: #define SO